In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# veride okunamayan karakterleri okutmak için encoding
df = pd.read_csv(r"C:\Users\ASUS\Desktop\Bitirme\2-encoding_analysis\encoded_data.csv") 

df.head(4)

x = df.drop(columns="price")
y = df["price"]

In [7]:
import statsmodels.api as sm

# Otomatik sabit sütunu (bias) ekle
X = np.append(arr=np.ones((x.shape[0], 1)).astype(int), values=x.values, axis=1)

# Başlangıçta tüm değişkenleri içeren bir model kur
X_opt = X[:, list(range(X.shape[1]))]  # Tüm sütunlar

# OLS modelini oluştur ve uygula
model = sm.OLS(y, X_opt).fit()

# Sonuç özeti
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.718
Method:                 Least Squares   F-statistic:                     7180.
Date:                Sun, 18 May 2025   Prob (F-statistic):               0.00
Time:                        22:00:02   Log-Likelihood:            -2.8274e+05
No. Observations:               16917   AIC:                         5.655e+05
Df Residuals:                   16910   BIC:                         5.656e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.822e+06   1.12e+05    -61.008      0.0

#### R-squared: 0.718 | Adj. R-squared: 0.718
Bu değerler modelin bağımlı değişken olan price'ı ne kadar açıkladığını gösterir.

%71.8'lik bir açıklama gücü oldukça güçlüdür. Demek ki model, fiyatı belirleyen değişkenleri oldukça iyi temsil ediyor.

#### Modelin Anlamlılığı
F-statistic: 7180 | Prob (F-statistic): 0.00
F istatistiği, modeldeki tüm değişkenlerin toplamda anlamlı bir katkı sağlayıp sağlamadığını test eder.

Prob(F-statistic) = 0.00 → Model genel olarak istatistiksel olarak anlamlı.


#### Bağımsız Değişkenlerin Anlamlılığı (P-value)

Tüm değişkenlerin p-değeri 0.05’in altında olduğu için hepsi modele anlamlı katkı sağlıyor. Şu anda backward elimination’a gerek görünmüyor.


#### Multicollinearity ve Sayısal Stabilite

Condition Number: 4.76e+07 Bu çok büyük bir değer.

Bu durum, değişkenler arasında yüksek korelasyon (multicollinearity) olabileceğini gösterir.

Multicollinearity varsa:

Katsayıların işaretleri/şiddetleri anlamsız hale gelebilir.

Model istikrarsız hale gelir.

Bu durumu netleştirmek için VIF (Variance Inflation Factor) analizi önerilir.


#### Hataların Normalliği ve Otokorelasyon

Durbin-Watson: 1.992 Bu değer 2'ye yakın → Otokorelasyon yok (iyi haber).

Omnibus, Jarque-Bera, Skew, Kurtosis Bu testler, artıkların (residuals) dağılımının normal olup olmadığını test eder.

Skew = 1.796, Kurtosis = 13.924 → Artıklar normal dağılmıyor.

Bu durum özellikle güven aralıkları ve bazı istatistiksel çıkarımlar için sorun oluşturabilir.

Gerekirse log dönüşümü veya Box-Cox dönüşümü gibi tekniklerle düzeltilebilir.


### 📌 Sonuç ve Öneriler
✔️ Model oldukça güçlü: Yüksek R² ve anlamlı değişkenler var.
⚠️ Multicollinearity riski var: Condition Number çok yüksek – VIF kontrolü yap.
⚠️ Artıklar normal dağılmıyor: Regresyon varsayımlarına %100 uyum yok.
🔁 İyileştirme için:

VIF analizi → çoklu doğrusal bağlantıyı kontrol et

Artıklar için dönüşüm (log, sqrt vb.) gerekebilir

Outlier kontrolü yap (boxplot vs.)

### VIF Analizi

VIF (Variance Inflation Factor), bir regresyon modelinde çoklu doğrusal bağlantıyı (multicollinearity) tespit etmek için kullanılır.

In [20]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split

X = df.drop(columns="price")
y = df["price"]

# Veriyi eğitim ve test olarak ayır (80% eğitim, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 'x' zaten price haricindeki bağımsız değişkenleri içeriyor
# Gerekirse normalizasyon yap (özellikle büyük sayılar varsa)
x_scaled = StandardScaler().fit_transform(x)

# VIF hesapla
vif_df = pd.DataFrame()
vif_df["Değişken"] = x.columns
vif_df["VIF Değeri"] = [variance_inflation_factor(x_scaled, i) for i in range(x_scaled.shape[1])]

print(vif_df)

           Değişken  VIF Değeri
0                m2    1.441840
1               age    1.022829
2             floor    1.078099
3        total_room    1.278974
4  district_encoded    2.557470
5  neighbor_encoded    2.661287
